In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image


In [2]:
# load the saved model
model = load_model('ARhinitisModel')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [3]:
# predicting images
# import urllib.request
# req = urllib.request.urlopen('http://answers.opencv.org/upfiles/logo_2.png')
# img = image.load_img(req, target_size=(img_width, img_height))
(img_width, img_height) = (224, 224)
img = image.load_img(r'C:/Users/cvpr/Documents/Bishal/Allergic Rhinitis/Dataset/non_rotate/dataset2/1/19_1_00002_right.tif', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images/255, batch_size=10)
print(np.argmax(classes, axis=1))
print(classes)

[1]
[[0.25708774 0.41965902 0.32325324]]
